The global schema for our application looks like- 
Customer (Name, email, phone#, loyaltyPts, Age Group, medical condition (optional))
Flight (flight#, model, economy seats,business seats, first class seats, price-eco,price-business,price- first class availability, status, departure_time, landing_time, source, destination, distance, delay)

We will be performing schema matching to identify the fields that match with our global schema. We wil be doing this through linguistic matching primarily


Importing schemas from databases

In [1]:
import psycopg2

In [ ]:
!pip install rapidfuzz

SELECT tablename
FROM pg_catalog.pg_tables
WHERE schemaname = 'your_schema_name';

In [2]:
from rapidfuzz import fuzz

def fuzzy_match(word, word_list):
    # Calculate similarity scores for each word in the list
    similarities = {w: fuzz.ratio(word, w) for w in word_list}
    return similarities



In [ ]:
results

In [ ]:
Customer = [
    "Name", "Email", "Contact number", "Points/frequent_flyer_miles", 
    "Date of Birth", "City", "Address"
]

Flight = [
    "Flight code", "Economy-Seats", "Business_Seats", "First Class-Seats", 
    "Economy Price", "Business Price", "First Class Price", 
    "Status", "Departure_Time", "Landing_Time", 
    "Source", "Destination", "Distance", "Delay"
]
global_schema = {
    "Customer": Customer,
    "Flight": Flight
}

#############################################

# Airline I
Customer_Info = ["CustomerID", "FullName", "EmailAddress", "ContactNumber", "LoyaltyPoints"]
Customer_Age = ["CustomerID", "AgeGroup"]  # Example: "18-25", "26-35", etc.
Customer_Health = ["CustomerID", "HasMedicalCondition"]  # Binary yes/no
Flight_Info = [
    "Flight_id", "FlightCode", "AircraftModel", "TotalSeats", 
    "TicketPrice-Economy", "TicketPrice-Business", "TicketPrice-First Class", 
    "AvailabilityStatus-Economy", "AvailabilityStatus-Business", "AvailabilityStatus-First Class", 
    "DepartureTime", "ArrivalTime"
]
Flight_Route = [
    "Flight_id", "FlightCode", "DepartureAirport", "DestinationAirport", 
    "FlightDistance", "CurrentStatus", "AverageDelay"
]

# Airline II
FlightDetails = [
    "FlightId", "FlightNumber", "AircraftType", "TotalCapacity", 
    "PricePerSeat-Economy", "PricePerSeat-Business", 
    "AvailableSeats-Economy", "AvailableSeats-Business", 
    "FlightStatus", "TakeoffTime", "LandTime"
]
Route = ["FlightId", "Origin", "Destination", "DistanceTraveled", "EstimatedDelay"]
Passenger = ["PassengerName", "Email", "Contact", "FrequentFlyerPoints", "DateOfBirth", "City"]

# Airline III
Clients = ["Full_Name", "EmailID", "ContactNo", "Age", "PreExistingCondition"]
Loyalty = ["ClientEmail", "Points"]
FlightSchedule = [
    "FlightID", "Flight code", "SeatsLeft", "FlightFare", 
    "Status", "StartTime", "EndTime"
]
RoutePlan = ["FlightID", "FromLocation", "ToLocation", "DistanceKM", "AvgDelay"]

# Consolidated view (if needed)
airline_schemas = {
    "Airline I": {
        "Customer":{
        "Customer_Info": Customer_Info,
        "Customer_Age": Customer_Age,
        "Customer_Health": Customer_Health,
        },
        "Flight":{
        "Flight_Info": Flight_Info,
        "Flight_Route": Flight_Route
        }
    },
    "Airline II": {
        "Flight":{
        "FlightDetails": FlightDetails,
        "Route": Route
        },
        "Customer":{
        
        "Passenger": Passenger
        }
    },
    "Airline III": {
        "Customer":{
        "Clients": Clients,
        "Loyalty": Loyalty
        },
        "Flight":{
        "FlightSchedule": FlightSchedule,
        "RoutePlan": RoutePlan
        }
    }
}


airline_schemas2=airline_schemas





In [3]:
import psycopg2

In [4]:
Connection_params={
    "air india":{
    "dbname": "air india",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "vistara":{
    "dbname": "vistara",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    },
    "indigo":{
    "dbname": "indigo",
    "user": "postgres",
    "password": "addy",
    "host": "localhost",  # Use the actual host (e.g., "127.0.0.1" or server IP)
    "port": 5432          # Default PostgreSQL port
    }
}

In [5]:
def custom_query(connection_params,query):
    try:
        # Establish connection
        conn = psycopg2.connect(**connection_params)
        print("Connection successful")

        # Create a cursor object
        cursor = conn.cursor()

        # Execute a query
        cursor.execute(f"{query}")

        # Fetch and print the result
        results= cursor.fetchall()
        # print(f"PostgreSQL version: {version}")
        column_names = [desc[0] for desc in cursor.description]
        # fields[connection_params["dbname"]]=column_names

        # Close cursor and connection
        cursor.close()
        conn.close()
        # print(results)
        return results
    except Exception as e:
        print(f"Error: {e}")
        return None

    

In [6]:
query="""SELECT table_name, column_name, data_type 
                            FROM information_schema.columns
                            WHERE table_schema = 'public'
                            ORDER BY table_name, ordinal_position;"""

# custom_query(Connection_params['Indigo'],query)

In [7]:
dbnames=["air india","indigo","vistara"]
all_schemas={}
for airline in dbnames:
    all_schemas[airline]={}
    result=custom_query(Connection_params[airline],query)
    for attribute in result:
        if attribute[0] not in all_schemas[airline]:
            all_schemas[airline][attribute[0]]=[attribute[1]]
        else:
            all_schemas[airline][attribute[0]].append(attribute[1])


Connection successful
Connection successful
Connection successful


In [ ]:
dbnames=["air india","indigo","vistara"]

In [73]:
attr_to_table={}
for db in dbnames:
    attr_to_table[db]={}
    for table,atts in all_schemas[db].items():
        for att in atts:
            attr_to_table[db][att]=table

In [74]:
attr_to_table

{'air india': {'CustomerID': 'customer_info',
  'FullName': 'customer_info',
  'EmailAddress': 'customer_info',
  'ContactNumber': 'customer_info',
  'LoyaltyPoints': 'customer_info',
  'dataofbirth': 'customer_info',
  'cityofresidence': 'customer_info',
  'address': 'customer_info',
  'flightid': 'flight_route',
  'flightcode': 'flight_route',
  'aircraftmodel': 'flight_info',
  'ticketprice_economy': 'flight_info',
  'ticketprice_business': 'flight_info',
  'ticketprice_firstclass': 'flight_info',
  'economy_seats': 'flight_info',
  'business_seats': 'flight_info',
  'firstclass_seats': 'flight_info',
  'departuretime': 'flight_info',
  'arrivaltime': 'flight_info',
  'departureairport': 'flight_route',
  'destinationairport': 'flight_route',
  'flightdistance': 'flight_route',
  'currentstatus': 'flight_route',
  'averagedelay': 'flight_route'},
 'indigo': {'flightid': 'routeplan',
  'flight_code': 'flightschedule',
  'seatsleft': 'flightschedule',
  'flightfare': 'flightschedule',

In [32]:
fuzzy_match("flight",["passenger"])

{'passenger': 13.33333333333333}

In [33]:
classify_tables={}

customer=set()
flight=set()
customer_syn=["passenger","customers"]
flight_syn=["flight","route"]
for db in dbnames:
    classify_tables[db]={"Customer":set(),"Flight":set()}
    for syn in customer_syn:
        res=fuzzy_match(syn,list(all_schemas[db].keys()))
        # print(type(res))
        for table,score in res.items():
            if score>43:
                classify_tables[db]["Customer"].add(table)
    
    for syn in flight_syn:
        res=fuzzy_match(syn,list(all_schemas[db].keys()))
        for table,score in res.items():
            if score>50:
                classify_tables[db]["Flight"].add(table)

print(classify_tables)
    
    



{'air india': {'Customer': {'customer_info'}, 'Flight': {'flight_info', 'flight_route'}}, 'indigo': {'Customer': {'traveller'}, 'Flight': {'routeplan', 'flightschedule'}}, 'vistara': {'Customer': {'passenger'}, 'Flight': {'flightdetails', 'route'}}}


field matching

In [ ]:
#global schema
Customer = [
   "Address", "Name", "Email", "Contact number", "Points", 
    "Date of Birth", "City"
]

Flight = [
    "Flight code", "Status", "Departure_Time", "Landing_Time", 
    "Source/From", "Destination", "Distance", "Delay","Economy Seats", "Business_Seats", "First Class-Seats", 
    "Economy Price", "Business Price", "First Class Price", 
]
global_schema = {
    "Customer": Customer,
    "Flight": Flight
}
combined_customer_fields={}
for db in dbnames:
    combined_customer_fields[db]=set()
    for table in classify_tables[db]["Customer"]:
        for field in all_schemas[db][table]:
            combined_customer_fields[db].add(field)

combined_flight_fields={}
for db in dbnames:
    combined_flight_fields[db]=set()
    for table in classify_tables[db]["Flight"]:
        for field in all_schemas[db][table]:
            combined_flight_fields[db].add(field)

all_schema2=all_schemas

cust_mappings={}
flight_mappings={}
for db in dbnames:
    cust_mappings[db]={}
    flight_mappings[db]={}
    for field in Customer:
        res=fuzzy_match(field,list(combined_customer_fields[db]))
        max_score=0
        max_field=""
        for field2,score in res.items():
            if score>max_score:
               max_score=score
               max_field=field2

        combined_customer_fields[db].discard(max_field)
        cust_mappings[db][field]=max_field

    for field in Flight:
        res=fuzzy_match(field,list(combined_flight_fields[db]))
        max_score=0
        max_field=""
        for field2,score in res.items():
            if score>max_score:
               max_score=score
               max_field=field2
        
        if max_score>20:
            combined_flight_fields[db].discard(max_field)
            flight_mappings[db][field]=max_field
        else:
            print(field,db)
        combined_flight_fields[db].discard(max_field)
        flight_mappings[db][field]=max_field
        
        
            


#for customer part



Business_Seats indigo
Economy Price indigo
Business Price indigo


In [75]:
cust_mappings

{'air india': {'Address': 'address',
  'Name': 'FullName',
  'Email': 'EmailAddress',
  'Contact number': 'ContactNumber',
  'Points': 'LoyaltyPoints',
  'Date of Birth': 'dataofbirth',
  'City': 'cityofresidence'},
 'indigo': {'Address': 'address',
  'Name': 'full_name',
  'Email': 'emailid',
  'Contact number': 'contactno',
  'Points': 'loyaltypoints',
  'Date of Birth': 'customerid',
  'City': 'city'},
 'vistara': {'Address': 'address',
  'Name': 'passengername',
  'Email': 'email',
  'Contact number': 'contact',
  'Points': 'loyaltypoints',
  'Date of Birth': 'dataofbirth',
  'City': 'city'}}

In [72]:
flight_mappings

{'air india': {'Flight code': 'flightcode',
  'Status': 'currentstatus',
  'Departure_Time': 'departuretime',
  'Landing_Time': 'arrivaltime',
  'Source/From': 'departureairport',
  'Destination': 'destinationairport',
  'Distance': 'flightdistance',
  'Delay': 'averagedelay',
  'Economy Seats': 'economy_seats',
  'Business_Seats': 'business_seats',
  'First Class-Seats': 'firstclass_seats',
  'Economy Price': 'ticketprice_economy',
  'Business Price': 'ticketprice_business',
  'First Class Price': 'ticketprice_firstclass'},
 'indigo': {'Flight code': 'flight_code',
  'Status': 'status',
  'Departure_Time': 'starttime',
  'Landing_Time': 'endtime',
  'Source/From': 'tolocation',
  'Destination': 'fromlocation',
  'Distance': 'distancekm',
  'Delay': 'avgdelay',
  'Economy Seats': 'seatsleft',
  'First Class-Seats': 'flightfare',
  'First Class Price': 'flightid'},
 'vistara': {'Flight code': 'flightid',
  'Status': 'flightstatus',
  'Departure_Time': 'landtime',
  'Landing_Time': 'take

In [ ]:
{'air india': {'Flight code': 'flightcode',
  'Status': 'currentstatus',
  'Departure_Time': 'departuretime',
  'Landing_Time': 'arrivaltime',
  'Source/From': 'departureairport',
  'Destination': 'destinationairport',
  'Distance': 'flightdistance',
  'Delay': 'averagedelay',
  'Economy Seats': 'economy_seats',
  'Business_Seats': 'business_seats',
  'First Class-Seats': 'firstclass_seats',
  'Economy Price': 'ticketprice_economy',
  'Business Price': 'ticketprice_business',
  'First Class Price': 'ticketprice_firstclass'},
 'indigo': {'Flight code': 'flight_code',
  'Status': 'status',
  'Departure_Time': 'starttime',
  'Landing_Time': 'endtime',
  'Source/From': 'fromlocation',
  'Destination': 'tolocation',
  'Distance': 'distancekm',
  'Delay': 'avgdelay',
  'Business_Seats indigo':'',
  'Business Price indigo':'',
  'Economy Price': 'flightfare',
  'Economy Seats': 'seatsleft',
  'First Class-Seats': '',
  'First Class Price': ''},
 'vistara': {'Flight code': 'flightid',
  'Status': 'flightstatus',
  'Departure_Time': 'takeofftime',
  'Landing_Time': 'landtime',
  'Source/From': 'origin',
  'Destination': 'destination',
  'Distance': 'distancetraveled',
  'Delay': 'estimateddelay',
  'Economy Seats': 'availableseats_economy',
  'Business_Seats': 'availableseats_business',
  'First Class-Seats': '',
  'Economy Price': 'priceperseat_economy',
  'Business Price': 'priceperseat_business',
  'First Class Price': ''}}

In [ ]:
mapping={}



for airlines in airline_schemas.values():
    customer_fields=[]
    flight_fields=[]
    internal={"Customer":customer_fields,"Flight":flight_fields}
    for cust_field in airlines["Customer"].values():
        customer_fields.extend(cust_field)
    for flight_field in airlines["Flight"].values():
        flight_fields.extend(flight_field)
    
    
    for k,v in global_schema.items():
        for field in v:
            result = fuzzy_match(field,internal[k])
            best_match=""
            best_score=0
            for word, score in result.items():
                if score>best_score:
                    best_match=word
                    best_score=score
            if field not in mapping:
                mapping[field]=[best_match]
                
            
            else:
                mapping[field].append(best_match)


            
        




In [ ]:
mapping